In [1]:
%load_ext autoreload
%autoreload 2

import sys
if sys.path[-1] != "..": sys.path.append("..")

from qBN.qBNMC import qBNMC
from qBN.qBNRejection import qBNRejection
from XPs.qBNRT import qRuntime

import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

import matplotlib.pyplot as plt

import numpy as np

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='1b6910ff55c1d3853e5c8e2ca2b0dbbc3b415fb897d26a6c272c63254527581c824aea1180585f706ab8263318f3c553549d136ca32952ef401abb54011eee33'
)

backend = service.get_backend("ibm_brisbane")

# Or save your credentials on disk.
# QiskitRuntimeService.save_account(channel='ibm_quantum', instance='ibm-q/open/main', token='1b6910ff55c1d3853e5c8e2ca2b0dbbc3b415fb897d26a6c272c63254527581c824aea1180585f706ab8263318f3c553549d136ca32952ef401abb54011eee33')


/tmp/ipykernel_22181/1844146419.py:9: DeprecationWarning: The get_backend() method is deprecated as of qiskit-ibm-runtime 0.24 and will be removed no sooner than 3 months after the release date. Please use backend() instead.
  backend = service.get_backend("ibm_brisbane")


In [3]:
bn1 = gum.fastBN("A->B<-C", 2)
bn1.cpt("A")[:] = [0.05, 0.95]
bn1.cpt("C")[:] = [0.05, 0.95]
bn1.cpt("B")[:] = [[[0.1, 0.9], [0.2, 0.8]], [[0.3, 0.7], [0.4, 0.6]]]
qbn1 = qBNMC(bn1)

In [4]:
ev1 = {"A": 0, "C":0}
tn1 = "B"
value_range = range(100,1000,100)

In [5]:
qinf1 = qBNRejection(qbn1)
qinf1.setEvidence(ev1)
qinf1.useFragmentBN(target={tn1})
qinf1.getGates()

In [6]:
ie1 = gum.LazyPropagation(bn1)
ie1.setEvidence(ev1)
ie1.makeInference()

In [7]:
nb_obs = 10

In [8]:
mc1_max_err = list()

for j in range(nb_obs):
    mc1_max_err.append(list())
    for i in value_range:

        mc1 = gum.MonteCarloSampling(bn1)
        mc1.setEvidence(ev1)
        mc1.setEpsilon(1e-10)
        mc1.setMaxTime(1e10)
        mc1.setMaxIter(i)
        mc1.makeInference()

        offset = mc1.posterior(tn1).toarray() - ie1.posterior(tn1).toarray()
        max_offset = offset.max()
        mc1_max_err[j].append(max_offset)

mc1_max_err = np.array(mc1_max_err)

In [9]:
qinf1_max_err = list()

for j in range(nb_obs):
    qinf1_max_err.append(list())
    print(f"Run number {j} -", end=' ')
    for i in value_range:

        qinf1.setMaxIter(i)
        qinf1.makeInference()

        offset = qinf1.posterior(tn1).toarray() - ie1.posterior(tn1).toarray()
        max_offset = offset.max()
        qinf1_max_err[j].append(max_offset)

        print(f"{i}: {max_offset:.4f}", end=', ')
    print()

qinf1_max_err = np.array(qinf1_max_err)

Run number 0 - 100: 0.0100, 200: 0.0400, 300: 0.0100, 400: 0.0100, 500: 0.0140, 600: 0.0117, 700: 0.0043, 800: 0.0112, 

KeyboardInterrupt: 

In [ ]:
mc1_mean = np.mean(mc1_max_err, axis=0)
mc1_percentile_25 = np.percentile(mc1_max_err, 25, axis=0)
mc1_percentile_75 = np.percentile(mc1_max_err, 75, axis=0)

qinf1_mean = np.mean(qinf1_max_err, axis=0)
qinf1_percentile_25 = np.percentile(qinf1_max_err, 25, axis=0)
qinf1_percentile_75 = np.percentile(qinf1_max_err, 75, axis=0)


plt.plot(value_range, mc1_mean, label='MC Mean', color='orange')
plt.plot(value_range, qinf1_mean, label='QI Mean', color='blue')

plt.fill_between(value_range, mc1_percentile_25, mc1_percentile_75, color='orange', alpha=0.2, label='25th-75th Percentile')
plt.fill_between(value_range, qinf1_percentile_25, qinf1_percentile_75, color='blue', alpha=0.2, label='25th-75th Percentile')

plt.xlabel('Iterations')
plt.ylabel('Error')
plt.title(f'Max Error evolution of MC/QI samplers ({nb_obs} observations)')
plt.legend()
plt.grid(True)

plt.savefig('ErrEvoIter.png')
plt.show()

In [ ]:
qrt1 = qRuntime(qinf1, backend)
qrt1.getGateExecutionTime()
A_time = qrt1.A_time
G_time = qrt1.G_time

In [ ]:
nb_obs = 10

In [ ]:
mc1_time = list()

for j in range(nb_obs):
    mc1_time.append(list())
    for i in value_range:

        mc1 = gum.MonteCarloSampling(bn1)
        mc1.setEvidence(ev1)
        mc1.setEpsilon(1e-10)
        mc1.setMaxTime(1e10)
        mc1.setMaxIter(i)
        mc1.makeInference()

        mc1_time[j].append(mc1.currentTime())

mc1_time = np.array(mc1_time)

In [ ]:
qinf1_time = list()

for j in range(nb_obs):
    qinf1_time.append(list())
    print(f"Run number {j} -", end=' ')
    for i in value_range:

        qinf1.setMaxIter(i)
        qinf1.makeInference()

        qrt = qRuntime(qinf1, backend)
        qrt.A_time = A_time
        qrt.G_time = G_time

        run_time = qrt.rejectionSamplingRuntime()
        qinf1_time[j].append(run_time)

        print(f"{i}: {run_time:.4f}", end=', ')
    print()

qinf1_time = np.array(qinf1_time)

In [ ]:
mc1_mean = np.mean(mc1_time, axis=0)
mc1_percentile_25 = np.percentile(mc1_time, 25, axis=0)
mc1_percentile_75 = np.percentile(mc1_time, 75, axis=0)

qinf1_mean = np.mean(qinf1_time, axis=0)
qinf1_percentile_25 = np.percentile(qinf1_time, 25, axis=0)
qinf1_percentile_75 = np.percentile(qinf1_time, 75, axis=0)


plt.plot(value_range, mc1_mean, label='MC Mean', color='orange')
plt.plot(value_range, qinf1_mean, label='QI Mean', color='blue')

plt.fill_between(value_range, mc1_percentile_25, mc1_percentile_75, color='orange', alpha=0.2, label='25th-75th Percentile')
plt.fill_between(value_range, qinf1_percentile_25, qinf1_percentile_75, color='blue', alpha=0.2, label='25th-75th Percentile')

plt.xlabel('Time')
plt.ylabel('Iterations')
plt.title(f'Time evolution of MC/QI samplers ({nb_obs} observations)')
plt.legend()
plt.grid(True)

plt.savefig('TimeEvoIter.png')
plt.show()

In [ ]:
value_range = range(100, 1001 ,100)
nb_obs = 10
iter = 200

In [ ]:

qinf_run_time_list = list()
qinf_proba_err_list = list()

for j in range(nb_obs):

    qinf_run_time_list.append(list())
    qinf_proba_err_list.append(list())

    for i in value_range:
        bn = gum.fastBN("A->B<-C", 2)
        bn.cpt("C")[:] = [i**(-1/2), 1-i**(-1/2)]
        bn.cpt("A")[:] = [i**(-1/2), 1-i**(-1/2)]
        bn.cpt("B")[:] = [[[0.3,0.7],[0.4,0.6]],[[0.5,0.5],[0.6,0.4]]]
        evidence = {"A": 0, "C": 0}
        target = "B"

        ie = gum.LazyPropagation(bn)
        ie.setEvidence(evidence)
        ie.makeInference()

        qbn = qBNMC(bn)

        qinf = qBNRejection(qbn)
        qinf.setEvidence(evidence)
        qinf.getGates()
        qinf.setMaxIter(iter)
        qinf.makeInference()

        qrt = qRuntime(qinf, backend)
        run_time = qrt.rejectionSamplingRuntime()

        error = (qinf.posterior(target).toarray() - ie.posterior(target).toarray()).max()

        print(f"i: {i}, P(e): {ie.evidenceProbability():.10f}, log: {qinf.log}, runtime: {run_time:.5f}, error: {error}")

        qinf_run_time_list[j].append(run_time)
        qinf_proba_err_list[j].append(error)

qinf_run_time_list = np.array(qinf_run_time_list)
qinf_proba_err_list = np.array(qinf_proba_err_list)

In [ ]:
mc_run_time_list = list()
mc_proba_err_list = list()

ev_proba_list = list()

for j in range(nb_obs):

    mc_run_time_list.append(list())
    mc_proba_err_list.append(list())
    ev_proba_list.append(list())

    for i in value_range:
        bn = gum.fastBN("A->B<-C", 2)
        bn.cpt("C")[:] = [i**(-1/2), 1-i**(-1/2)]
        bn.cpt("A")[:] = [i**(-1/2), 1-i**(-1/2)]
        bn.cpt("B")[:] = [[[0.3,0.7],[0.4,0.6]],[[0.5,0.5],[0.6,0.4]]]
        evidence = {"A": 0, "C": 0}
        target = "B"

        ie = gum.LazyPropagation(bn)
        ie.setEvidence(evidence)
        ie.makeInference()

        mc = gum.MonteCarloSampling(bn)

        mc.setEvidence(evidence)
        mc.setEpsilon(1e-10)
        mc.setMaxTime(1e10)
        mc.setMaxIter(iter)
        mc.makeInference()

        run_time = mc.currentTime()

        error = (mc.posterior(target).toarray() - ie.posterior(target).toarray()).max()

        print(f"i: {i}, P(e): {ie.evidenceProbability():.10f}, runtime: {run_time:.5f}, error: {error}")

        mc_run_time_list[j].append(run_time)
        mc_proba_err_list[j].append(error)

        ev_proba_list[j].append(ie.evidenceProbability())

ev_proba_list = ev_proba_list[0]

mc_run_time_list = np.array(mc_run_time_list)
mc_proba_err_list = np.array(mc_proba_err_list)

In [ ]:
mc1_mean = np.mean(mc_run_time_list, axis=0)
mc1_percentile_25 = np.percentile(mc_run_time_list, 25, axis=0)
mc1_percentile_75 = np.percentile(mc_run_time_list, 75, axis=0)

qinf1_mean = np.mean(qinf_run_time_list, axis=0)
qinf1_percentile_25 = np.percentile(qinf_run_time_list, 25, axis=0)
qinf1_percentile_75 = np.percentile(qinf_run_time_list, 75, axis=0)

plt.plot(ev_proba_list, mc1_mean, label='MC Mean', color='orange')
plt.plot(ev_proba_list, qinf1_mean, label='QI Mean', color='blue')

plt.fill_between(ev_proba_list, mc1_percentile_25, mc1_percentile_75, color='orange', alpha=0.2, label='25th-75th Percentile')
plt.fill_between(ev_proba_list, qinf1_percentile_25, qinf1_percentile_75, color='blue', alpha=0.2, label='25th-75th Percentile')

plt.xlabel('Evidence probability')
plt.ylabel('Time')
plt.title(f'Time evolution of MC/QI samplers ({nb_obs} observations, {iter} iterations)')
plt.legend()
plt.grid(True)

plt.savefig('TimeEvoProb.png')
plt.show()

In [ ]:
mc1_mean = np.mean(mc_proba_err_list, axis=0)
mc1_percentile_25 = np.percentile(mc_proba_err_list, 25, axis=0)
mc1_percentile_75 = np.percentile(mc_proba_err_list, 75, axis=0)

qinf1_mean = np.mean(qinf_proba_err_list, axis=0)
qinf1_percentile_25 = np.percentile(qinf_proba_err_list, 25, axis=0)
qinf1_percentile_75 = np.percentile(qinf_proba_err_list, 75, axis=0)

plt.plot(ev_proba_list, mc1_mean, label='MC Mean', color='orange')
plt.plot(ev_proba_list, qinf1_mean, label='QI Mean', color='blue')

plt.fill_between(ev_proba_list, mc1_percentile_25, mc1_percentile_75, color='orange', alpha=0.2, label='25th-75th Percentile')
plt.fill_between(ev_proba_list, qinf1_percentile_25, qinf1_percentile_75, color='blue', alpha=0.2, label='25th-75th Percentile')

plt.xlabel('Evidence probability')
plt.ylabel('Error')
plt.title(f'Error evolution of MC/QI samplers ({nb_obs} observations, {iter} iterations)')
plt.legend()
plt.grid(True)

plt.savefig('ErrEvoProb.png')
plt.show()